# Part 1

--- Day 9: Encoding Error ---

With your neighbor happily enjoying their video game, you turn your attention to an open data port on the little screen in the seat in front of you.

Though the port is non-standard, you manage to connect it to your computer through the clever use of several paperclips. Upon connection, the port outputs a series of numbers (your puzzle input).

The data appears to be encrypted with the eXchange-Masking Addition System (XMAS) which, conveniently for you, is an old cypher with an important weakness.

XMAS starts by transmitting a preamble of 25 numbers. After that, each number you receive should be the sum of any two of the 25 immediately previous numbers. The two numbers will have different values, and there might be more than one such pair.

For example, suppose your preamble consists of the numbers 1 through 25 in a random order. To be valid, the next number must be the sum of two of those numbers:

- 26 would be a valid next number, as it could be 1 plus 25 (or many other pairs, like 2 and 24).
- 49 would be a valid next number, as it is the sum of 24 and 25.
- 100 would not be valid; no two of the previous 25 numbers sum to 100.
- 50 would also not be valid; although 25 appears in the previous 25 numbers, the two numbers in the pair must be different.

Suppose the 26th number is 45, and the first number (no longer an option, as it is more than 25 numbers ago) was 20. Now, for the next number to be valid, there needs to be some pair of numbers among 1-19, 21-25, or 45 that add up to it:

- 26 would still be a valid next number, as 1 and 25 are still within the previous 25 numbers.
- 65 would not be valid, as no two of the available numbers sum to it.
- 64 and 66 would both be valid, as they are the result of 19+45 and 21+45 respectively.

Here is a larger example which only considers the previous 5 numbers (and has a preamble of length 5):

```
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
```

In this example, after the 5-number preamble, almost every number is the sum of two of the previous 5 numbers; the only number that does not follow this rule is 127.

The first step of attacking the weakness in the XMAS data is to find the first number in the list (after the preamble) which is not the sum of two of the 25 numbers before it. What is the first number that does not have this property?

In [1]:
(import [pathlib [Path]])

In [2]:
(setv INPUT_FILE (/ (.cwd Path) "inputs" "day09" "part1.txt"))

In [3]:
(defn input-lines []
  (with [fp (.open INPUT_FILE)]
    (yield-from (gfor line fp (.strip line)))))

In [4]:
(defn input-ints []
  (yield-from (gfor line (input-lines) (int line))))

In [24]:
(defn sliding-window [num iterable]
  (setv seq (list iterable))
  (setv start 0)
  (setv end num)
  (while (< end (len seq))
    (yield (cut seq start end))
    (+= start 1)
    (+= end 1)))

In [18]:
(defn sliding-window2 [num iterable]
  (setv seq (list iterable))
  (yield-from
    (gfor
      [start end]
      (zip (count) (range num (len seq)))
      (cut seq start end))))

In [14]:
(defn combinations [seq num]
  (unless (= num 0)
    (yield (, (first seq) #* (combinations (rest seq) (dec num))))))

In [16]:
(list (combinations [1 2 3 4 5] 3))

[(1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 3, 4), (1, 3, 5), (1, 4, 5), (2, 3, 4), (2, 3, 5), (2, 4, 5), (3, 4, 5)]

In [19]:
(defn do-sum? [target numbers num]
  (any (gfor combos (combinations numbers num) (= (sum combos) target))))

In [30]:
(defn find-wrong [numbers preamble-size]
  (first
    (gfor
      [target nums]
      (zip (islice numbers preamble-size None) (sliding-window2 preamble-size numbers))
      :if (not (do-sum? target nums 2))
      target)))

In [31]:
(find-wrong (list (input-ints)) 25)

32321523

# Part 2

The final step in breaking the XMAS encryption relies on the invalid number you just found: you must find a contiguous set of at least two numbers in your list which sum to the invalid number from step 1.

Again consider the above example:

```
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
```

In this list, adding up all of the numbers from 15 through 40 produces the invalid number from step 1, 127. (Of course, the contiguous set of numbers in your actual list might be much longer.)

To find the encryption weakness, add together the smallest and largest number in this contiguous range; in this example, these are 15 and 47, producing 62.

What is the encryption weakness in your XMAS-encrypted list of numbers?

In [32]:
(setv TARGET-VAL 32321523)

In [33]:
(defn do-sum-min-max [seq target]
  (when (= (sum seq) target)
    (, (min seq) (max seq))))

In [35]:
(defn one-size [numbers window-size target]
  (some
    identity
    (gfor
      window
      (sliding-window2 window-size numbers)
      (do-sum-min-max window target))))

In [38]:
(, (one-size [1 2 3 4 5] 3 6) (one-size [1 2 3 4 5] 3 90))

((1, 3), None)

In [39]:
(defn find-window [numbers target]
  (some
    identity
    (gfor
      window-size
      (range 2 500)
      (one-size numbers window-size target))))

In [40]:
(find-window (list (input-ints)) TARGET-VAL)

(1145730, 3649251)

In [41]:
(sum (, 1145730 3649251))

4794981